In [2]:
# Setup environment and dependencies
try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv
    
    load_dotenv()
except ImportError:
    pass

In [ ]:
# Initialize Google Gemini model
from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [ ]:
# Test the model with a simple message
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Explain few-shot prompting in a couple of short sentences."),
]

response = model.invoke(messages)
print(response.content)

Few-shot prompting is a technique where you provide a large language model with a few examples of the task you want it to perform, directly within the prompt itself. This helps the model understand the desired input-output format and underlying pattern, leading to better performance on new, similar tasks without requiring any model fine-tuning.


In [ ]:
# Import components for few-shot prompting
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Create output parser
parser = StrOutputParser()

print("Components imported successfully!")

Components imported successfully!


In [ ]:
# Create examples for few-shot prompting
# Examples for sentiment analysis
examples = [
    {"input": "I love this movie! It's amazing!", "output": "Positive"},
    {"input": "This food tastes terrible.", "output": "Negative"},
    {"input": "The weather is okay today.", "output": "Neutral"},
    {"input": "I'm so excited about my vacation!", "output": "Positive"},
    {"input": "This service is disappointing.", "output": "Negative"}
]

print("Examples created:")
for i, example in enumerate(examples, 1):
    print(f"{i}. Input: '{example['input']}' → Output: '{example['output']}'")

Examples created:
1. Input: 'I love this movie! It's amazing!' → Output: 'Positive'
2. Input: 'This food tastes terrible.' → Output: 'Negative'
3. Input: 'The weather is okay today.' → Output: 'Neutral'
4. Input: 'I'm so excited about my vacation!' → Output: 'Positive'
5. Input: 'This service is disappointing.' → Output: 'Negative'


In [ ]:
# Create the few-shot prompt template
# Define the example template
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}")
])

# Create the few-shot prompt template
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# Test the few-shot prompt
formatted_examples = few_shot_prompt.format()
print("Few-shot examples formatted:")
print(formatted_examples)

Few-shot examples formatted:
Human: I love this movie! It's amazing!
AI: Positive
Human: This food tastes terrible.
AI: Negative
Human: The weather is okay today.
AI: Neutral
Human: I'm so excited about my vacation!
AI: Positive
Human: This service is disappointing.
AI: Negative


In [ ]:
# Create the complete prompt template with examples
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a sentiment analyzer. Classify the sentiment of the given text as Positive, Negative, or Neutral. Look at these examples:"),
    few_shot_prompt,
    ("human", "Now classify this: {input}")
])

print("Complete prompt template created!")

Complete prompt template created!


In [ ]:
# Create and test the few-shot chain
# Create the chain
sentiment_chain = final_prompt | model | parser

# Test with new examples
test_inputs = [
    "This book is incredible!",
    "I hate waiting in long lines.",
    "The meeting was fine.",
    "Best pizza I've ever had!"
]

print("Testing few-shot prompting:")
print("=" * 40)

for text in test_inputs:
    result = sentiment_chain.invoke({"input": text})
    print(f"Input: '{text}'")
    print(f"Output: {result}")
    print("-" * 20)

Testing few-shot prompting:
Input: 'This book is incredible!'
Output: Positive
--------------------
Input: 'I hate waiting in long lines.'
Output: Negative
--------------------
Input: 'The meeting was fine.'
Output: Neutral
--------------------
Input: 'Best pizza I've ever had!'
Output: Positive
--------------------
